In [1]:
!pip -q install scikit-learn pandas numpy


In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline


In [3]:
DATA_DIR = "/content/init_data"

train = pd.read_csv(f"{DATA_DIR}/train_chat.csv")
test  = pd.read_csv(f"{DATA_DIR}/test_chat.csv")
sample = pd.read_csv(f"{DATA_DIR}/sample_submission_chat.csv")

train.head()


,message_id,text,day,hour,topic
0,msg_000001,ТСЖ подняли тариф? в платёжке +300₽ 😁,14,0,bills
1,msg_000002,"Соседи по дому, отдам даром чайник, вдруг кому...",11,19,market
2,msg_000003,"Миша, ты делал блины? как получилось?",9,20,cooking
3,msg_000004,"нашли ошейник у почтовых ящиков, без адресника...",8,14,pets
4,msg_000005,"Салют, в подъезде мигает свет у мусорки, как н...",10,21,repairs


In [4]:
def build_text(df):
    return (
        df["text"].fillna("") +
        " <DAY_" + df["day"].astype(str) + ">" +
        " <HOUR_" + df["hour"].astype(str) + ">"
    )

train["full_text"] = build_text(train)
test["full_text"]  = build_text(test)


In [5]:
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=3,
    max_df=0.9,
    analyzer="word",
    sublinear_tf=True
)

clf = LogisticRegression(
    C=6,
    max_iter=3000,
    class_weight="balanced",
    n_jobs=-1
)

pipe = Pipeline([
    ("tfidf", tfidf),
    ("clf", clf)
])


In [6]:
X = train["full_text"]
y = train["topic"]

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores = []

for fold, (tr_idx, val_idx) in enumerate(skf.split(X, y), 1):
    X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

    pipe.fit(X_tr, y_tr)
    preds = pipe.predict(X_val)

    f1 = f1_score(y_val, preds, average="macro")
    scores.append(f1)

    print(f"Fold {fold}: Macro-F1 = {f1:.4f}")

print("Mean Macro-F1:", np.mean(scores))


Fold 1: Macro-F1 = 0.9999
Fold 2: Macro-F1 = 0.9998
Fold 3: Macro-F1 = 1.0000
Fold 4: Macro-F1 = 1.0000
Fold 5: Macro-F1 = 0.9999
Mean Macro-F1: 0.9999250849735504


In [7]:
pipe.fit(train["full_text"], train["topic"])


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.9, min_df=3, ngram_range=(1, 2),
                                 sublinear_tf=True)),
                ('clf',
                 LogisticRegression(C=6, class_weight='balanced', max_iter=3000,
                                    n_jobs=-1))])

In [8]:
test_preds = pipe.predict(test["full_text"])

submission = pd.DataFrame({
    "message_id": test["message_id"],
    "topic": test_preds
})

submission.to_csv("submission.csv", index=False)
submission.head()


,message_id,topic
0,msg_120001,congratulations
1,msg_120002,bills
2,msg_120003,cleaning
3,msg_120004,plans_guests
4,msg_120005,lost_found
